### 어텐션
- 입력 시퀀스의 모든 요소를 동일하게 처리하는 대신에 입력 단어의 중요한 부분에 가중치를 부여해 번역 품질을 향상시키는 매커니즘

1. 가중치 할당
    - 쿼리, 키, 값을 통해 계산
2. 동적 컨텍스트 생성
    - 가중치가 적용된 입력의 가중합을 동적으로 생성

- Transformer Attention 종류
    - Encoder self-attention: globalSelfAttention
        - 쿼리, 키, 값 모두 인코더 상태를 사용
    - decoder self-attention: causalAttention
        - 쿼리, 키, 값 모두 디코더 상태를 사용
    - encoder-decoder attention: crossAttention
        - 쿼리는 디코더의 출력을, 키,값은 인코더 상태를 사용

### Input Feeding
- 디코더에서 이전 시간 단계의 어텐션 정보와 현재 입력을 결합해 더 풍부한 문맥 정보를 제공하는 기법
- 주요 개념: 문맥 정보 강화(이전 어텐션 정보를 활용해 번역의 일관성과 정확성을 높임), 작동방식(현재 입력과 이전 컨텍스트 벡터(어텐션 정보)를 결합하고, 디코더가 문맥 정보를 누적 학습)

### 전이 학습
1. 사전 훈련 방식
    - 정적인 단어 임베딩을 사용해 단어를 고정된 벡터로 표현하는 기법
    - 주요 알고리즘
        - word2vec: cbow(주변 단어로 중심 단어 예측), skipgram: 중심 단어로 주변 단어 예측
        - glove: 단어 공기행렬을 기반으로 임베딩 학습
        - fasttext: 단어를 ngram으로 분해해 희소 단어 처리 강화
2. ELMo
    - 문맥 기반 단어 임베딩으로 문맥에 따라 단어의 의미를 동적으로 표현
    - 특징
        - 양방향 lstm 사용
3. Transformers
    - 병렬 처리와 전역 문맥 이해를 통해 rnn의 한계를 극복한 딥러닝 모델로, 인코더-디코더 구조와 어텐션 매커니즘을 기반으로 함
    - 주요 구성 요소
        - 인코더: 입력 시퀀스를 벡터로 변환
        - 디코더: 인코더의 벡터를 사용해 출력 시퀀스 생성
        - multi-head-attention: 단어 간 관계를 다각도로 학습
        - 포지셔널 인코딩: 단어 순서 정보 추가
4. BERT
5. GPT-2

----

# Attention

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### 어텐션 가중치 계산

In [19]:
def attention(query, key, value):
    # 1. 어텐션 스코어 계산 (Query - Key)
    scores = torch.matmul(query, key.transpose(-2,-1))
    print('Attention Score Shape: ', scores.shape)

    # 2. Softmax 적용 (가중치 계산)
    attention_weights = F.softmax(scores, dim=1)
    print('Attention weights shape: ', attention_weights.shape)

    # 3. 어텐션 밸류 계산 (Value 적용 => 최종 Context vector 계산)
    context_vector = torch.matmul(attention_weights, value)
    print('Context Vector shape: ', context_vector.shape)

    return context_vector

In [20]:
# 토큰화 및 임베딩 결과 예시
vocab = {
    '나는': 0,
    '학교에': 1,
    '간다': 2,
    '<pad>': 3
}

In [21]:
vocab_size = len(vocab)
EMBEDDING_DIM = 4

In [22]:
# 입력 문장
inputs = ['나는', '학교에', '간다']
inputs_ids = torch.tensor([[vocab[word] for word in inputs]])   # (1,3) # 배치 차원 맞추기 위해 2번 리스트로 감싸줌

In [23]:
# 1. 임베딩 적용
embedding_layer = nn.Embedding(vocab_size, EMBEDDING_DIM)
inputs_embedded = embedding_layer(inputs_ids)       # 임베딩 레이어 통과
# print(inputs_embedded.shape)  # torch.Size([1, 3, 4])

# 2. 선형 변환 -> Query, Key, Value
HIDDEN_DIM = 4
W_query = nn.Linear(EMBEDDING_DIM, HIDDEN_DIM)
W_key = nn.Linear(EMBEDDING_DIM, HIDDEN_DIM)
W_value = nn.Linear(EMBEDDING_DIM, HIDDEN_DIM)

input_query = W_query(inputs_embedded)
input_key = W_key(inputs_embedded)
input_value = W_value(inputs_embedded)

print(input_query.shape, input_key.shape, input_value.shape)

torch.Size([1, 3, 4]) torch.Size([1, 3, 4]) torch.Size([1, 3, 4])


In [24]:
context_vector = attention(input_query, input_key, input_value)
context_vector

Attention Score Shape:  torch.Size([1, 3, 3])
Attention weights shape:  torch.Size([1, 3, 3])
Context Vector shape:  torch.Size([1, 3, 4])


tensor([[[ 0.1480,  0.4819,  0.2151, -0.8311],
         [-0.1020,  0.1860, -0.0669, -0.3845],
         [-0.7443,  0.0393, -0.6395, -0.4993]]], grad_fn=<UnsafeViewBackward0>)

### seq2seq 모델에 어텐션 추가

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
    
    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.shape[1]
        hidden_expanded = hidden.unsqueeze(1).repeat(1, seq_len, 1)     # 입력 시퀀스만큼 복제
        energy = torch.tanh(self.attn(torch.cat((hidden_expanded, encoder_outputs), dim=2)))    # 인코더 출력을 붙임 # tanh를 통과시켜 비선형으로
        attention_scores = torch.sum(self.v * energy, dim=2)        # 현재 상태에서 가중치 곱하고 합계를 구해서 스칼라값으로 표현
        attention_weights = F.softmax(attention_scores, dim=1)      # attention 가중치
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        
        return context_vector, attention_weights

In [34]:
class Seq2SeqWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2SeqWithAttention, self).__init__()
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.decoder_input_transform = nn.Linear(input_dim, hidden_dim)
    
    def forward(self, encoder_input, decoder_input):
        encoder_outputs, hidden = self.encoder(encoder_input)
        context_vector, _ = self.attention(hidden[-1], encoder_outputs)
        decoder_input_ = self.decoder_input_transform(decoder_input)
        output, _ = self.decoder(decoder_input_, hidden)
        combined = torch.cat((output, context_vector.unsqueeze(1)), dim=2)
        
        return self.fc(combined)

In [ ]:
batch_size = 1
seq_len = 5
input_dim = 10
hidden_dim = 20
output_dim = 15

# 랜덤하게 encoder, decoder input 만들기
encoder_input = torch.randn(batch_size, seq_len, input_dim)
decoder_input = torch.randn(batch_size, 1, input_dim)

model = Seq2SeqWithAttention(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
result = model(encoder_input, decoder_input)
print(result)

tensor([[[-0.1539,  0.0579, -0.0606,  0.0322,  0.0160, -0.1993,  0.0144,
           0.0858,  0.1954, -0.1625,  0.1705,  0.0152, -0.1274,  0.0169,
           0.0659]]], grad_fn=<ViewBackward0>)
